In [67]:
from init import *

In [68]:
pd.options.display.max_columns

100

In [69]:
%time train = pd.read_csv("train_ver2.csv")
display(train.head())
display(train.shape)

/home/raischel/Envs/ds1/lib64/python3.10/site-packages/IPython/core/magics/execution.py:1335: DtypeWarning: Columns (5,8,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code, glob, local_ns)


CPU times: user 32.8 s, sys: 24.1 s, total: 56.8 s
Wall time: 1min 50s


,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,NaN,1.0,A,S,N,NaN,KHL,N,1.0,29.0,MALAGA,1.0,87218.10,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,NaN,1.0,I,S,S,NaN,KHE,N,1.0,13.0,CIUDAD REAL,0.0,35548.74,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,NaN,1.0,I,S,N,NaN,KHE,N,1.0,13.0,CIUDAD REAL,0.0,122179.11,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,NaN,1.0,I,S,N,NaN,KHD,N,1.0,50.0,ZARAGOZA,0.0,119775.54,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,NaN,1.0,A,S,N,NaN,KHE,N,1.0,50.0,ZARAGOZA,1.0,NaN,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0


(13647309, 48)

In [70]:
train_uid = set(train.ncodpers.unique())

In [71]:
len(train_uid)

956645

In [72]:
train.fecha_dato.value_counts()

2016-05-28    931453
2016-04-28    928274
2016-03-28    925076
2016-02-28    920904
2016-01-28    916269
2015-12-28    912021
2015-11-28    906109
2015-10-28    892251
2015-09-28    865440
2015-08-28    843201
2015-07-28    829817
2015-06-28    632110
2015-05-28    631957
2015-04-28    630367
2015-03-28    629209
2015-02-28    627394
2015-01-28    625457
Name: fecha_dato, dtype: int64

In [73]:
train_fdm = train.fecha_dato.max()
train_fdm

'2016-05-28'

# Modified find target: add last date

In [138]:
def df2target(df,uid):
    """ 
    converts a competition data frame filteredd for a given uid into a target string 
    with the submissionformat:
    
    must return df of format
    ncodpers,added_products
    15889,ind_tjcr_fin_ult1
    15890,ind_tjcr_fin_ult1 ind_recibo_ult1
    15892,ind_nomina_ult1
    15893,
    etc.
    """
    #dfi = df.ncodpers==uid#.isin(ncodpers)
    dfi=df[df.ncodpers==uid].sort_values("fecha_dato",  ascending=True)
    #dfi.sort("fecha_dato", inplace=True, ascending=True)
    dfid=dfi[target_cols].diff(axis=0)
    dfid = dfid.iloc[-1:,:]#[dfid.fecha_dato==data_max]
    a=dfid[dfid >0].stack()#.index.tolist()
    a=a.reset_index()["level_1"].unique().tolist()
    a=",".join([str(uid),]+a)
    return a

In [150]:
#test_public_data = []
#for uid in list(test_public_uid)[101:111]:
#test_public_data.append(df2target(test, uid))# , test.fecha_dato.max()))
#%lprun df2target(test, uid)
def testrun():
    test_public_data = []
    for uid in list(test_public_uid)[101:131]:
        test_public_data.append(df2target(test, uid))# , test.fecha_dato.max()))
%prun -s cumulative  -l 10 testrun()        

In [ ]:
%%time
test_public_data = []
for uid in list(test_public_uid):
    test_public_data.append(df2target(test, uid))# , test.fecha_dato.max()))

In [142]:
type(a)

NoneType

In [130]:
test_public_data

['15891',
 '15892',
 '15895',
 '15897',
 '15902',
 '15906',
 '15907',
 '15908',
 '15910',
 '15911',
 '15913',
 '15920',
 '15922',
 '15923',
 '15924',
 '15929,ind_recibo_ult1',
 '15930',
 '15932',
 '15933',
 '15936',
 '15937',
 '15938',
 '15941',
 '15948',
 '15950',
 '15952,ind_recibo_ult1',
 '15954',
 '15956',
 '15957',
 '15958',
 '15960',
 '15962',
 '15969',
 '15972',
 '15974',
 '15975',
 '15979',
 '15985',
 '15990',
 '15991',
 '15993,ind_tjcr_fin_ult1',
 '15994',
 '15998',
 '15999',
 '16001',
 '16004',
 '16005',
 '16007',
 '16008,ind_tjcr_fin_ult1',
 '16009',
 '16010',
 '16018',
 '16024',
 '16027',
 '16028',
 '16029',
 '16033',
 '16036',
 '16038',
 '16041',
 '16042',
 '16047',
 '16048',
 '16049',
 '16053',
 '16056,ind_nomina_ult1,ind_nom_pens_ult1',
 '16067',
 '16074',
 '16075',
 '16078',
 '16083',
 '16085',
 '16093',
 '16095',
 '16098',
 '16100',
 '16103',
 '16105',
 '16110',
 '16111',
 '16115',
 '16119',
 '16126',
 '16131',
 '16136',
 '16137',
 '16141',
 '16145',
 '16147',
 '16152'

In [126]:
#5% taxa
#for i in range(100):
#    uid = list(test_public_uid)[i]
#    #uid
#    dfi=test[test.ncodpers==uid].sort_values("fecha_dato",  ascending=True)
#    dfid=dfi[target_cols].diff(axis=0)
#    dfid = dfid.iloc[-1:,:]#.reset_index()
#    a=dfid[dfid >0].stack()
#    display(a)

Series([], dtype: float64)

Series([], dtype: float64)

Series([], dtype: float64)

Series([], dtype: float64)

Series([], dtype: float64)

...

Series([], dtype: float64)

Series([], dtype: float64)

Series([], dtype: float64)

Series([], dtype: float64)

Series([], dtype: float64)

# Split Train / Test Set

In [75]:
#!Not yet!!
#test = train[train.fecha_dato==train_fdm].copy()
test=train.copy()

In [76]:
display(test.shape)
display(test.fecha_dato.max())

(13647309, 48)

'2016-05-28'

In [77]:
display(train.shape)
display(train.fecha_dato.max())
train = train[train.fecha_dato<train_fdm]
display(train.shape)
display(train.fecha_dato.max())


(13647309, 48)

'2016-05-28'

(12715856, 48)

'2016-04-28'

## check that all test UIDs are a subset of train UIDS

In [78]:
train_uid = set(train.ncodpers.unique())
display(len(train_uid))
test_uid = set(test.ncodpers.unique())
display(len(test_uid))
display(test_uid.issubset(train_uid))

951952

956645

False

In [79]:
len(test_uid.intersection(train_uid))

951952

In [80]:
new_test_uid = test_uid.intersection(train_uid)
test = test[test.ncodpers.isin(new_test_uid)]
display(test.shape)

(13642616, 48)

In [81]:
#test again 
train_uid = set(train.ncodpers.unique())
display(len(train_uid))
test_uid = set(test.ncodpers.unique())
display(len(test_uid))
display(test_uid.issubset(train_uid))

951952

951952

True

# Create raindom split train private / public 

In [82]:
help(input)

Help on method raw_input in module ipykernel.kernelbase:

raw_input(prompt='') method of ipykernel.ipkernel.IPythonKernel instance
    Forward raw_input to frontends
    
    Raises
    ------
    StdinNotImplementedError if active frontend doesn't support stdin.



In [83]:
rs = int(input());

1234


In [84]:
np.random.seed(rs)
test_private_uid = np.random.choice(list(new_test_uid), size=int(len(new_test_uid)/2), replace=False)

In [85]:
test_public_uid = new_test_uid.difference(test_private_uid)

In [86]:
display(len(test_public_uid))
display(len(test_private_uid))

475976

475976

In [87]:
463380+463380

926760

In [88]:
test_public_uid.intersection(test_private_uid)

set()

# Create solution  from test set 

In [64]:
test_public_data = []
for uid in list(test_public_uid):
    test_public_data.append(df2target(train, uid))

In [66]:
test_public_data[:100]

['15890',
 '15892,ind_cco_fin_ult1,ind_dela_fin_ult1',
 '15895',
 '15897,ind_nom_pens_ult1',
 '15900,ind_cno_fin_ult1',
 '15902',
 '15908',
 '15913,ind_ctop_fin_ult1,ind_cco_fin_ult1,ind_ecue_fin_ult1,ind_valo_fin_ult1',
 '15914,ind_nomina_ult1,ind_nom_pens_ult1',
 '15915',
 '15916,ind_cno_fin_ult1',
 '15919',
 '15920,ind_valo_fin_ult1',
 '15922',
 '15923,ind_tjcr_fin_ult1,ind_reca_fin_ult1',
 '15925,ind_recibo_ult1,ind_tjcr_fin_ult1',
 '15927,ind_cco_fin_ult1,ind_tjcr_fin_ult1',
 '15928,ind_tjcr_fin_ult1',
 '15929,ind_recibo_ult1',
 '15930',
 '15933',
 '15936',
 '15937,ind_tjcr_fin_ult1,ind_cno_fin_ult1',
 '15940',
 '15942',
 '15943,ind_recibo_ult1',
 '15944,ind_cco_fin_ult1',
 '15945',
 '15947',
 '15948',
 '15949,ind_recibo_ult1',
 '15950,ind_cco_fin_ult1',
 '15952',
 '15954',
 '15956',
 '15958,ind_ctpp_fin_ult1',
 '15960',
 '15964,ind_reca_fin_ult1',
 '15967,ind_ecue_fin_ult1',
 '15969,ind_cco_fin_ult1',
 '15973',
 '15974',
 '15975,ind_tjcr_fin_ult1,ind_fond_fin_ult1',
 '15980',
 '1

# Remove target columsn from test set 

# Write all to disk 